## Librerias

In [300]:
import pandas as pd
import numpy as np
from flask import Flask, request 
from pymongo import MongoClient
import requests
import markdown.extensions.fenced_code

## Datos en Base de datos para Mongo

In [301]:
!ls

Data                                  config
Notebook Mongo_df & Calling_API.ipynb miapi.py
Notebook Sentiment_Analysis.ipynb     tools
README.md


In [302]:
data = pd.read_fwf("Data/SW_EpisodeIV.txt",header=None)
data

,0,1,2,3,4,5,6,7,8,9
0,"""character"" ""dialogue""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""1"" ""THREEPIO"" ""Did you hear that? They've sh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""2"" ""THREEPIO"" ""We're doomed!""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""3"" ""THREEPIO"" ""There'll be no escape for the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""4"" ""THREEPIO"" ""What's that?""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1006,"""1006"" ""LUKE"" ""Oh, no!""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,"""1007"" ""THREEPIO"" ""Oh, my! Artoo! Can you he...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,"""1008"" ""TECHNICIAN"" ""We'll get to work on him ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009,"""1009"" ""THREEPIO"" ""You must repair him! Sir, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
del data[1]
del data[2]
del data[3]
del data[4]
del data[5]
del data[6]
del data[7]
del data[8]
del data[9]

In [304]:
data = data.iloc[1:]
data

,0
1,"""1"" ""THREEPIO"" ""Did you hear that? They've sh..."
2,"""2"" ""THREEPIO"" ""We're doomed!"""
3,"""3"" ""THREEPIO"" ""There'll be no escape for the ..."
4,"""4"" ""THREEPIO"" ""What's that?"""
5,"""5"" ""THREEPIO"" ""I should have known better tha..."
...,...
1006,"""1006"" ""LUKE"" ""Oh, no!"""
1007,"""1007"" ""THREEPIO"" ""Oh, my! Artoo! Can you he..."
1008,"""1008"" ""TECHNICIAN"" ""We'll get to work on him ..."
1009,"""1009"" ""THREEPIO"" ""You must repair him! Sir, ..."


In [305]:
sw_data = data[0].str.split(" ", n = 2, expand = True)
sw_data.rename(columns={0:'Line_order', 1:'Character', 2:'Dialogue'}, inplace=True)
sw_data.Character = sw_data.Character.apply(lambda x: x.strip('"'))
sw_data.Line_order = sw_data.Line_order.apply(lambda x: x.strip('"'))
sw_data.Dialogue = sw_data.Dialogue.apply(lambda x: x.strip('"'))

In [306]:
sw_data

,Line_order,Character,Dialogue
1,1,THREEPIO,Did you hear that? They've shut down the main...
2,2,THREEPIO,We're doomed!
3,3,THREEPIO,There'll be no escape for the Princess this time.
4,4,THREEPIO,What's that?
5,5,THREEPIO,I should have known better than to trust the l...
...,...,...,...
1006,1006,LUKE,"Oh, no!"
1007,1007,THREEPIO,"Oh, my! Artoo! Can you hear me? Say somethi..."
1008,1008,TECHNICIAN,We'll get to work on him right away.
1009,1009,THREEPIO,"You must repair him! Sir, if any of my circui..."


Transform to Json

In [307]:
sw_data.to_json("Data/SW_EpisodeIV.json", orient="records")

## Llamando API

Getting Characters:

In [310]:
url = "http://localhost:5000/characters"
data = requests.get(url).json()
data

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /characters (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118bb5c10>: Failed to establish a new connection: [Errno 61] Connection refused'))

Getting Dialogue from Character:

In [312]:
url_query_character = "http://localhost:5000/dialogue/"
person = "VADER"
dialogue = requests.get(url_query_character + person)
dialogue

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /dialogue/VADER (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118cd72b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

Posting new dialogue for character in specific line:

In [316]:
url_query_newpost = "http://localhost:5000/new_dialogue/"
datos = {
"Line_order" : "800",
"Character" : "VADER",
"Dialogue" : "Luke, I left for tabaco, and thats why you haven't seen me in a while"
}
new_post = requests.post(url_query_newpost, data = datos)
new_post

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /line_order/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118cd7fd0>: Failed to establish a new connection: [Errno 61] Connection refused'))